# Get Dark Area Values from Across SSA

In [1]:
import ee
import geemap
import pandas as pd

## Authenticate & Initialize GEE

Requires a [Google Cloud Project](https://console.cloud.google.com/projectcreate) and to enable the [Earth Engine API](https://console.cloud.google.com/apis/api/earthengine.googleapis.com) for the project. Find detailed instructions [here](https://book.geemap.org/chapters/01_introduction.html#earth-engine-authentication).

In [2]:
ee.Initialize()

## Create a GEEMap Object

In [7]:
m = geemap.Map(
    center=[-5, 15], 
    zoom=3, 
    basemap = 'Esri.WorldImagery',
    height = 1200
)

## Add Layers to the Map

In [11]:
# add nightlights median
# https://developers.google.com/earth-engine/datasets/catalog/NOAA_VIIRS_DNB_MONTHLY_V1_VCMSLCFG
dataset_night = ee.ImageCollection('NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG') \
                  .filter(ee.Filter.date('2020-01-01', '2024-01-01'))
nighttime = dataset_night.select('avg_rad')
image_night = nighttime.median()
nighttimeVis = {'min': 0.0, 'max': 2.0}
m.addLayer(image_night, nighttimeVis, 'Nighttime', True)


In [14]:
# add latest world pop data layer
# https://developers.google.com/earth-engine/datasets/catalog/WorldPop_GP_100m_pop
dataset_pop = ee.ImageCollection('WorldPop/GP/100m/pop') \
                  .filter(ee.Filter.date('2020-01-01', '2024-01-01'))
pop = dataset_pop.select('population')
image_pop = pop.median()
popVis = {'min': 0.0, 'max': 20.0, 'palette': ['24126c', '1fff4f', 'd4ff50']}
m.addLayer(image_pop, popVis, 'Population', True)

In [27]:
# add DarkMatter country labels to basemap
m.add_basemap('CartoDB.VoyagerOnlyLabels')

In [8]:
# show map
m

Map(center=[-5, 15], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(chil…

## Save Down & Read in Ocean, Desert, and Jungle Features

In [ ]:
# save the ocean polygon drawn on the map to a ee.Feature object
ocean_feat = m.draw_last_feature
# export the ee.Feature object to a geojson
geemap.ee_to_geojson(ocean_feat, 'data/dark_africa/ocean.geo.json')


In [ ]:
# save desert polygon
desert_feat = m.draw_last_feature
geemap.ee_to_geojson(desert_feat, 'data/dark_africa/desert.geo.json')

In [ ]:
# jungle poly
jungle_feat = m.draw_last_feature
geemap.ee_to_geojson(jungle_feat, 'data/dark_africa/jungle.geo.json')

In [15]:
# read back in geojson files to ee features
ocean_feat = geemap.geojson_to_ee('data/dark_africa/ocean.geo.json')
desert_feat = geemap.geojson_to_ee('data/dark_africa/desert.geo.json')
jungle_feat = geemap.geojson_to_ee('data/dark_africa/jungle.geo.json')

In [16]:
# add the features to the map
m.addLayer(ocean_feat, {'color': 'blue'}, 'ocean')
m.addLayer(desert_feat, {'color': 'brown'}, 'desert')
m.addLayer(jungle_feat, {'color': 'darkgreen'}, 'jungle')

## Get Countries with Mini-Grids and Polygons

In [36]:
# get list of countries in CLUB-ER dataset
cluber_df = pd.read_csv('data/cluber/cluber_sites_clean.csv')
cbil_df = pd.read_csv('data/cbil/site_data.csv')
pg_df = pd.read_csv('data/pg/site_data.csv')

# covert date_commissioned to date
cluber_df['date_commissioned'] = pd.to_datetime(cluber_df['date_commissioned'])
# filter out sites commissioned before 2014-01-01
cluber_df = cluber_df[cluber_df['date_commissioned'] >= '2014-01-01']

# get a unique list of countries
cluber_countries = cluber_df['country'].unique()
print('Club-ER Countries: ', cluber_countries)
cbil_countries = cbil_df['country'].unique()
print('Cross Boundary Countries: ', cbil_countries)
pg_countries = pg_df['country'].unique()
print('PowerGen Countries: ', pg_countries)

# combine the three lists of countries into one with just unique values
all_minigrid_countries = list(set(cluber_countries) | set(cbil_countries) | set(pg_countries))
# remove nan values
all_minigrid_countries = [x for x in all_minigrid_countries if str(x) != 'nan']
print('All Countries', all_minigrid_countries)
# export the list of countries to a csv
pd.DataFrame(all_minigrid_countries, columns=['country']).to_csv('data/dark_africa/countries.csv', index=False)


Club-ER Countries:  ['Angola' 'Burkina Faso' 'Cameroon' 'DR Congo' 'Ethiopia' 'Ghana' 'Kenya'
 'Liberia' 'Madagascar' 'Mali' 'Mauritania' 'Senegal' 'Tanzania' 'Togo'
 'Zambia' 'Zimbabwe']
Cross Boundary Countries:  ['Kenya' 'Sierra Leone' 'DR Congo' 'Nigeria' 'Tanzania' 'Haiti' 'Zambia'
 nan 'Ghana' 'Mozambique' 'Benin']
PowerGen Countries:  ['Nigeria' 'Sierra Leone' 'Tanzania']
All Countries ['Mozambique', 'Togo', 'DR Congo', 'Cameroon', 'Mauritania', 'Haiti', 'Zimbabwe', 'Benin', 'Burkina Faso', 'Sierra Leone', 'Madagascar', 'Tanzania', 'Senegal', 'Mali', 'Liberia', 'Ethiopia', 'Angola', 'Kenya', 'Zambia', 'Ghana', 'Nigeria']


In [37]:
# modify list of countries

# exclude Haiti
all_minigrid_countries = [x for x in all_minigrid_countries if x != 'Haiti']

# rename DR Congo to Democratic Republic of the Congo
all_minigrid_countries = [x if x != 'DR Congo' else 'Democratic Republic of the Congo' for x in all_minigrid_countries]

# rename Tanzania to United Republic of Tanzania
all_minigrid_countries = [x if x != 'Tanzania' else 'United Republic of Tanzania' for x in all_minigrid_countries]

In [38]:
# overlay country boundaries with white borders
countries = ee.FeatureCollection('FAO/GAUL/2015/level0')
style = {'color': 'ffffffff', 'width': 2, 'lineType': 'solid', 'opacity': 1}
# m.addLayer(countries, style, 'Countries', False)
# print the ADM0_NAME property of the countries to the console
print(countries.aggregate_array('ADM0_NAME').getInfo())



['Montenegro', 'Serbia', 'South Sudan', 'Sudan', 'Taiwan', 'Cocos (Keeling) Islands', 'Christmas Island', 'Ashmore and Cartier Islands', 'Faroe Islands', 'Mayotte', 'Réunion', 'Tromelin Island', 'Juan de Nova Island', 'Glorioso Island', 'Europa Island', 'Bassas da India', 'Saint Pierre et Miquelon', 'French Southern and Antarctic Territories', 'Denmark', 'The former Yugoslav Republic of Macedonia', 'Croatia', 'Malta', 'San Marino', 'Slovenia', 'Greece', 'Italy', 'Portugal', 'Spain', 'Bosnia and Herzegovina', 'Andorra', 'Albania', 'Monaco', 'Netherlands', 'Luxembourg', 'Liechtenstein', 'France', 'Germany', 'Switzerland', 'Belgium', 'Austria', 'Australia', 'Palau', 'Canada', 'Canada', 'Canada', 'Mozambique', 'Mauritius', 'Malawi', 'Rwanda', 'Somalia', 'Zambia', 'Kenya', 'Madagascar', 'Seychelles', 'United Republic of Tanzania', 'Uganda', 'Zimbabwe', 'Ethiopia', 'Eritrea', 'Djibouti', 'Comoros', 'Burundi', 'Gabon', 'Equatorial Guinea', 'Chad', 'Sao Tome and Principe', 'Congo', 'Democratic

In [39]:

# create a fc of just the countries in all_minigrid_countries
all_minigrid_countries_fc = countries.filter(ee.Filter.inList('ADM0_NAME', all_minigrid_countries))
m.addLayer(all_minigrid_countries_fc, style, 'Countries', True)

# note: this isn't styling the countries correctly
# the "fillColor" parameter doesn't seem to work

# count the number of countries in all_minigrid_countries_fc
all_minigrid_countries_fc.size()
print('countries included in the map: ', all_minigrid_countries_fc.aggregate_array('ADM0_NAME').getInfo())

countries included in the map:  ['Mozambique', 'Zambia', 'Kenya', 'Madagascar', 'United Republic of Tanzania', 'Zimbabwe', 'Ethiopia', 'Democratic Republic of the Congo', 'Cameroon', 'Angola', 'Togo', 'Sierra Leone', 'Senegal', 'Nigeria', 'Mauritania', 'Mali', 'Liberia', 'Ghana', 'Burkina Faso', 'Benin']


In [ ]:
# add place names
# https://developers.google.com/earth-engine/datasets/catalog/FAO_GAUL_2015_level0
m.add_basemap('CartoDB.DarkMatterOnlyLabels')
# m.remove_layer('CartoDB.VoyagerOnlyLabels') # doesn't work

## Get the Landcover Data from Google Earth Engine

In [21]:
# pull in a global high resolution land cover dataset
# https://developers.google.com/earth-engine/datasets/catalog/ESA_WorldCover_v200
landcover = ee.ImageCollection('ESA/WorldCover/v200').first()

landcover_africa = landcover.clip(all_minigrid_countries_fc)

visualization = {
  'bands': ['Map'],
}

print(landcover_africa.select('Map').getInfo())

m.addLayer(landcover_africa, visualization, 'Landcover', False)

# # inspect this image
# print(landcover_africa.getInfo())
# # inspect the bands of landcover_africa
# print(landcover_africa.bandNames().getInfo())
# # inspect the values of the band 'Map'

{'type': 'Image', 'bands': [{'id': 'Map', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}, 'dimensions': [4320000, 1728000], 'crs': 'EPSG:4326', 'crs_transform': [8.333333333333333e-05, 0, -180, 0, -8.333333333333333e-05, 84]}], 'version': 1699537784392512, 'id': 'ESA/WorldCover/v200/2021', 'properties': {'Map_class_names': ['Tree cover', 'Shrubland', 'Grassland', 'Cropland', 'Built-up', 'Bare / sparse vegetation', 'Snow and ice', 'Permanent water bodies', 'Herbaceous wetland', 'Mangroves', 'Moss and lichen'], 'system:time_start': 1609455600000, 'system:time_end': 1640991600000, 'Map_class_palette': ['006400', 'ffbb22', 'ffff4c', 'f096ff', 'fa0000', 'b4b4b4', 'f0f0f0', '0064c8', '0096a0', '00cf75', 'fae6a0'], 'Map_class_values': [10, 20, 30, 40, 50, 60, 70, 80, 90, 95, 100], 'system:asset_size': 109661138988, 'system:index': '2021'}}


## Define Landcover Classes of Interest

Value	Color	Description
10	#006400	Tree cover
20	#ffbb22	Shrubland
30	#ffff4c	Grassland
40	#f096ff	Cropland
50	#fa0000	Built-up
60	#b4b4b4	Bare / sparse vegetation
70	#f0f0f0	Snow and ice
80	#0064c8	Permanent water bodies
90	#0096a0	Herbaceous wetland
95	#00cf75	Mangroves
100	#fae6a0	Moss and lichen

#### Values to Extract
- 10: Tree cover --> this should be dark, but not as dark as 60. Will include forests.
- 20: Shrubland --> this should be darker than 30, but not as dark as 10. Will include savannahs.
- 30: Grassland --> this should be the brightest of the vegetated areas. Will include grasslands.
- 40: Cropland --> this should be brighter than 10, but not as bright as 30. Will include farmlands.
- 50: Built-up --> this should be bright, brighter than anything else hopefully. Will include bright cities.
- 60: Bare / sparse vegetation --> this should mostly be desert, hopefully the darkest.

#### Values to Skip
- 70: Snow and ice --> there isn't much at all int he continent
- 80: Permanent water bodies --> I should get this from my ocean polygon
- 90: Herbaceous wetland --> also don't know how much of this there is
- 95: Mangroves --> there are some, but not in most countries

## Select Just for Dark Areas that are far away from Built Areas

In [22]:

# create a mask for the desired landcovers
built_mask = landcover_africa.eq(50)
built = landcover_africa.updateMask(built_mask)
# m.addLayer(built, {'palette': 'red'}, 'Built')

# create a 10km buffer around built areas
built_buffer = built.focal_max(10000, 'circle', 'meters')
# m.addLayer(built_buffer, {'palette': 'red'}, 'Built buffer', False)

# create a mask for the built_buffer
# need to unmask it to convert areas outside of mask from nodata to 0
rural_mask = built_buffer.eq(50).unmask(0).eq(0)
# select areas outside of the built buffer in countries of interest
rural = landcover_africa.updateMask(rural_mask)
# m.addLayer(rural, {'palette': 'brown'},  'Rural landcover', False)



## Sample Rural, Built, Ocean, Desert, and Jungle Areas

### Rural

In [ ]:
# sample rural landcover
rural_pts = rural.sample(
    region=all_minigrid_countries_fc,
    scale=1000,
    numPixels=3000,
    seed=44,
    projection='EPSG:4326',
    geometries=True,
    dropNulls=True
)

# add a property "type" to the built_pts feature collection equal to "built"
rural_pts = rural_pts.map(lambda f: f.set('type', 'rural'))

# export rural sample to geojson
geemap.ee_to_geojson(rural_pts, 'data/dark_africa/rural_pts.geo.json')

### Built

In [23]:
built_pts = built.sample(
    region=all_minigrid_countries_fc,
    scale=1000,
    numPixels=10000,
    seed=44,
    projection='EPSG:4326',
    geometries=True,
    dropNulls=True
)

# add a property "type" to the built_pts feature collection equal to "built"
built_pts = built_pts.map(lambda f: f.set('type', 'built'))

# export built sample to geojson
geemap.ee_to_geojson(built_pts, 'data/dark_africa/built_pts.geo.json')

### Ocean

In [ ]:
# sample ocean polygon
ocean_pts = landcover.sample(
    region=ocean_feat,
    scale=1000,
    numPixels=1000,
    seed=44,
    projection='EPSG:4326',
    geometries=True,
    dropNulls=False
)

# add a property "type" to the ocean_pts feature collection equal to "ocean"
ocean_pts = ocean_pts.map(lambda f: f.set('type', 'ocean'))

# export sample to geojson
geemap.ee_to_geojson(ocean_pts, 'data/dark_africa/ocean_pts.geo.json')

### Desert

In [ ]:
# sample ocean polygon
desert_pts = landcover.sample(
    region=desert_feat,
    scale=1000,
    numPixels=1000,
    seed=44,
    projection='EPSG:4326',
    geometries=True,
    dropNulls=True
)

# add a property "type" to the desert_pts feature collection equal to "desert"
desert_pts = desert_pts.map(lambda f: f.set('type', 'desert'))

# export sample to geojson
geemap.ee_to_geojson(desert_pts, 'data/dark_africa/desert_pts.geo.json')

### Jungle

In [ ]:
# sample ocean polygon
jungle_pts = landcover.sample(
    region=jungle_feat,
    scale=1000,
    numPixels=1000,
    seed=44,
    projection='EPSG:4326',
    geometries=True,
    dropNulls=True
)

# add a property "type" to the jungle_pts feature collection equal to "jungle"
jungle_pts = jungle_pts.map(lambda f: f.set('type', 'jungle'))

# export sample to geojson
geemap.ee_to_geojson(jungle_pts, 'data/dark_africa/jungle_pts.geo.json')

### Read Back in Points and Add to Map

In [24]:
# read in the sample points geojsons
built_pts = geemap.geojson_to_ee('data/dark_africa/built_pts.geo.json')
rural_pts = geemap.geojson_to_ee('data/dark_africa/rural_pts.geo.json')
ocean_pts = geemap.geojson_to_ee('data/dark_africa/ocean_pts.geo.json')
desert_pts = geemap.geojson_to_ee('data/dark_africa/desert_pts.geo.json')
jungle_pts = geemap.geojson_to_ee('data/dark_africa/jungle_pts.geo.json')

# add the sample points to the map with appropriate colors
m.addLayer(built_pts, {'color': 'red'}, 'Built points')
m.addLayer(rural_pts, {'color': 'purple'}, 'Rural points')
m.addLayer(ocean_pts, {'color': 'blue'}, 'Ocean points')
m.addLayer(desert_pts, {'color': 'brown'}, 'Desert points')
m.addLayer(jungle_pts, {'color': 'darkgreen'}, 'Jungle points')

In [ ]:
# rural_trees_mask = rural.eq(10)
# rural_shrub_mark = rural.eq(20)
# rural_grass_mask = rural.eq(30)
# rural_crop_mask = rural.eq(40)
# rural_bare_mask = rural.eq(60)

# rural_trees = rural.updateMask(rural_trees_mask)
# rural_shrub = rural.updateMask(rural_shrub_mark)
# rural_grass = rural.updateMask(rural_grass_mask)
# rural_crop = rural.updateMask(rural_crop_mask)
# rural_bare = rural.updateMask(rural_bare_mask)

# note this drops any points that have been masked
# rural_trees_points = rural_rural.sample(
#     region=all_minigrid_countries_fc.geometry(),
#     scale=30, # 1000km
#     numPixels=1, # 10k points, many get dropped
#     seed=44,
#     dropNulls=True, # drop any points that have been masked
#     geometries=True
# )
# # convert into a feature collection of points
# rural_rural_fc = ee.FeatureCollection(rural_trees_points)

# save the feature collections
# geemap.ee_export_vector(rural_trees_fc, 'data/dark_africa/rural_trees_points.geojson')


## Create a 1km Buffer Around the Sampled Points

In [ ]:
# read in the feature collections from geojson
built_pts_fc = geemap.geojson_to_ee('data/dark_africa/built_pts.geo.json')
ocean_pts_fc = geemap.geojson_to_ee('data/dark_africa/ocean_pts.geo.json')
desert_pts_fc = geemap.geojson_to_ee('data/dark_africa/desert_pts.geo.json')
jungle_pts_fc = geemap.geojson_to_ee('data/dark_africa/jungle_pts.geo.json')

# combine the feature collections
dark_pts_fc = built_pts_fc.merge(ocean_pts_fc).merge(desert_pts_fc).merge(jungle_pts_fc)

In [ ]:
# create a 1km buffer around the rural water points
dark_pts_buffer = dark_pts_fc.map(lambda f: f.buffer(1000))


## Get the Nightlights Values for each buffer

In [ ]:
# get the image collectino of nightlight images
dataset_night = ee.ImageCollection('NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG') \
                  .filter(ee.Filter.date('2014-01-01', '2024-01-01'))
# select the avg_rad band
nighttime_ic = dataset_night.select('avg_rad')
# stack the images into a single image
nighttime_ic_stack = nighttime_ic.toBands()
nighttime_ic_stack

In [ ]:
# apply the reduceRegions for dark
nighttime_dark_fc = nighttime_ic_stack.reduceRegions(
    collection=dark_pts_buffer, 
    reducer=ee.Reducer.mean(), 
    scale=50
)


In [ ]:
# get the centerpoint of each polygon feature in the feature collection
nighttime_dark_fc = nighttime_dark_fc.map(lambda f: f.set('center', f.geometry().centroid().coordinates()))

## Convert the Feature Collections to GeoDataFrames and Clean Data

In [ ]:
# convert fc to gdf
nighttime_dark_gdf = geemap.ee_to_geopandas(nighttime_dark_fc)
# nighttime_dark_gdf.head(5)

In [ ]:
# remove "_avg_rad" from the column names that contain it
nighttime_dark_gdf.columns = nighttime_dark_gdf.columns.str.replace('_avg_rad', '')
nighttime_dark_gdf.head(6)

In [ ]:
# export the geodataframe to a csv
nighttime_dark_gdf.to_csv('data/dark_africa/nighttime_dark_gdf.csv', index=False)

##############################################################################################################################################################################
##############################################################################################################################################################################
##############################################################################################################################################################################
##############################################################################################################################################################################
##############################################################################################################################################################################
##############################################################################################################################################################################
clean  
pull into R  
compare rural points by map class  
compare to built points  
compare to ocean juungle and desert points  
comapre to mini-grids  
compare to control sites  

## Read in GDF, Clean, and Melt

In [ ]:
import pandas as pd

# read in the csvs as geodataframes
nighttime_rural_df = pd.read_csv('data/dark_africa/nighttime_rural_gdf.csv')

# view head
nighttime_rural_df.head(5)


In [ ]:
# drop extra columns 'geometry' if they exist
nighttime_rural_df = nighttime_rural_df.drop(columns=['geometry'])
# add a column "type" that is equal to "rural"
nighttime_rural_df['type'] = 'rural'

# set id column to be equal to the row number
nighttime_rural_df['id'] = nighttime_rural_df.index

nighttime_rural_df.head(5)




In [ ]:

# melt the data into long format
nighttime_rural_melt_df = nighttime_rural_df.melt(id_vars=['id', 'center', 'type', 'Map'], var_name='image_date', value_name='image_value')

nighttime_rural_melt_df.head(5)

In [ ]:
# export the melted dataframes to csv
nighttime_rural_melt_df.to_csv('data/dark_africa/nighttime_rural_melt_df.csv', index=False)